# Question Answering using Embeddings

Many use cases require GPT-3 to respond to user questions with insightful answers. For example, a customer support chatbot may need to provide answers to common questions. The GPT models have picked up a lot of general knowledge in training, but we often need to ingest and use a large library of more specific information.

In this notebook we will demonstrate a method for enabling GPT-3 able to answer questions using a library of text as a reference, by using document embeddings and retrieval.

In [1]:
import numpy as np
import openai
import pandas as pd
import pickle
import tiktoken
import time

openai.api_key = '<Key>'

COMPLETIONS_MODEL = "text-davinci-003"
EMBEDDING_MODEL = "text-embedding-ada-002"

In [2]:
# We have hosted the processed dataset, so you can download it directly without having to recreate it.
# This dataset has already been split into sections, one row for each section of the Wikipedia page.

df = pd.read_csv('cleaned_data.csv')
df = df.set_index(["sectionNumber", "sectionTitle"])
print(f"{len(df)} rows in the data.")
df.sample(5)

600 rows in the data.


,,content,tokens
sectionNumber,sectionTitle,,
SectionNumber: 366-A.,Procuration of minor girl:,366-A.\nProcuration of minor girl: Procuratio...,190
SectionNumber: 247,Fraudulently or dishonestly diminishing weight or altering composition of Pakistan coin:,247\nFraudulently or dishonestly diminishing w...,88
SectionNumber: 80,Accident in doing a lawful act:,80\nAccident in doing a lawful act: Accident ...,108
SectionNumber: 70,"Fine leviable within six years, or during imprisonment; Death not to discharge property from liability:","70\nFine leviable within six years, or during ...",128
SectionNumber: 181,False statement on oath or affirmation to public servant or person authorised to administer an oath or affirmation:,181\nFalse statement on oath or affirmation to...,141


In [3]:
def get_embedding(text: str, model: str=EMBEDDING_MODEL) -> list[float]:
    result = openai.Embedding.create(
      model=model,
      input=text
    )
    time.sleep(5)
    return result["data"][0]["embedding"]

def compute_doc_embeddings(df: pd.DataFrame) -> dict[tuple[str, str], list[float]]:
    """
    Create an embedding for each row in the dataframe using the OpenAI Embeddings API.
    
    Return a dictionary that maps between each embedding vector and the index of the row that it corresponds to.
    """
    return {
        idx: get_embedding(r.content) for idx, r in df.iterrows()
    }

In [4]:
def load_embeddings(fname: str) -> dict[tuple[str, str], list[float]]:
    """
    Read the document embeddings and their keys from a CSV.
    
    fname is the path to a CSV with exactly these named columns: 
        "title", "heading", "0", "1", ... up to the length of the embedding vectors.
    """
    
    df = pd.read_csv(fname, header=0)
    max_dim = max([int(c) for c in df.columns if c != "sectionNumber" and c != "sectionTitle"])
    return {
           (r.sectionNumber, r.sectionTitle): [r[str(i)] for i in range(max_dim + 1)] for _, r in df.iterrows()
    }

In [5]:
document_embeddings = load_embeddings("modifiedJupyter.csv")

# ===== OR, uncomment the below line to recaculate the embeddings from scratch. ========

# document_embeddings = compute_doc_embeddings(df)
# embeddings_df = pd.DataFrame(document_embeddings)
# embeddings_df.to_csv('jupyterGeneratedEmbeddings.csv', index=False)

C:\Users\Shabeeb\AppData\Local\Temp\ipykernel_15380\1919024425.py:9: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(fname, header=0)


In [6]:
# An example embedding:
example_entry = list(document_embeddings.items())[0]
print(f"{example_entry[0]} : {example_entry[1][:5]}... ({len(example_entry[1])} entries)")

('SectionNumber: 1', 'Title and extent of operation of the Code.') : [-0.005502332, -0.000821574, 0.001789243, -0.0202365, -0.005426823]... (1536 entries)


In [7]:
def vector_similarity(x: list[float], y: list[float]) -> float:
    """
    Returns the similarity between two vectors.
    
    Because OpenAI Embeddings are normalized to length 1, the cosine similarity is the same as the dot product.
    """
    return np.dot(np.array(x), np.array(y))

def order_document_sections_by_query_similarity(query: str, contexts: dict[(str, str), np.array]) -> list[(float, (str, str))]:
    """
    Find the query embedding for the supplied query, and compare it against all of the pre-calculated document embeddings
    to find the most relevant sections. 
    
    Return the list of document sections, sorted by relevance in descending order.
    """
    query_embedding = get_embedding(query)
    
    document_similarities = sorted([
        (vector_similarity(query_embedding, doc_embedding), doc_index) for doc_index, doc_embedding in contexts.items()
    ], reverse=True)
    
    return document_similarities

In [8]:
order_document_sections_by_query_similarity("What is the definition of a 'Judge?'", document_embeddings)[:5]

[(0.8820595149666943, ('SectionNumber: 19', '"Judge".')),
 (0.8584503039643534, ('SectionNumber: 20', '"Court of Justice".')),
 (0.8406330091247475,
  ('SectionNumber: 77', 'Act of Judge when acting judicially:')),
 (0.800627371011219,
  ('SectionNumber: 219',
   'Public servant in judicial proceeding corruptly making report, etc., contrary to law:')),
 (0.8003102436349301,
  ('SectionNumber: 229', 'Personation of a Juror or assessor:'))]

In [9]:
order_document_sections_by_query_similarity('What is the difference between "fraudulently" and "defraud"?', document_embeddings)[:5]

[(0.8658651053528361, ('SectionNumber: 25', '"Fraudulently".')),
 (0.8171067258899785,
  ('SectionNumber: 207',
   'Fraudulent claim to property to prevent its seizure as forfeited or in execution:')),
 (0.8138158907905736,
  ('SectionNumber: 477',
   'Fraudulent cancellation, destruction, etc., of will, authority to adopt, or valuable security:')),
 (0.8135823567644422,
  ('SectionNumber: 206',
   'Fraudulent removal or concealment of property to prevent its seizure as forfeited or in execution:')),
 (0.8117879787518463,
  ('SectionNumber: 208', 'Fraudulently suffering decree for sum not due:'))]

In [10]:
MAX_SECTION_LEN = 500
SEPARATOR = "\n* "
ENCODING = "cl100k_base"  # encoding for text-embedding-ada-002

encoding = tiktoken.get_encoding(ENCODING)
separator_len = len(encoding.encode(SEPARATOR))

f"Context separator contains {separator_len} tokens"

'Context separator contains 3 tokens'

In [11]:
def construct_prompt(question: str, context_embeddings: dict, df: pd.DataFrame) -> str:
    """
    Fetch relevant 
    """
    most_relevant_document_sections = order_document_sections_by_query_similarity(question, context_embeddings)
#     print(most_relevant_document_sections)
    chosen_sections = []
    chosen_sections_len = 0
    chosen_sections_indexes = []
     
    for _, section_index in most_relevant_document_sections:
        # Add contexts until we run out of space.        
#         print(section_index)
        document_section = df.loc[section_index]
        print(int(document_section.tokens))
        chosen_sections_len += int(document_section.tokens) + separator_len
#         chosen_sections_len += document_section.tokens + separator_len
#         print(document_section.tokens.item())
        if chosen_sections_len > MAX_SECTION_LEN:
            break
            
        chosen_sections.append(SEPARATOR + str(document_section.content.replace("\n", " ")))
        chosen_sections_indexes.append(str(section_index))
            
    # Useful diagnostic information
    print(f"Selected {len(chosen_sections)} document sections:")
    print("\n".join(chosen_sections_indexes))
    
    header = """Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text below, say "I don't know."\n\nContext:\n"""
    
    return header + "".join(chosen_sections) + "\n\n Q: " + question + "\n A:"

In [12]:
prompt = construct_prompt(
    'What is the difference between "fraudulently" and "defraud"?',
    document_embeddings,
    df
)

print("===\n", prompt)

41
196
155
177
Selected 3 document sections:
('SectionNumber: 25', '"Fraudulently".')
('SectionNumber: 207', 'Fraudulent claim to property to prevent its seizure as forfeited or in execution:')
('SectionNumber: 477', 'Fraudulent cancellation, destruction, etc., of will, authority to adopt, or valuable security:')
===
 Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text below, say "I don't know."

Context:

* sectionNumber      sectionTitle   
SectionNumber: 25  "Fraudulently".    25\n"Fraudulently".  "Fraudulently". A person ...
Name: content, dtype: object
* sectionNumber       sectionTitle                                                                     
SectionNumber: 207  Fraudulent claim to property to prevent its seizure as forfeited or in execution:    207\nFraudulent claim to property to prevent i...
Name: content, dtype: object
* sectionNumber       sectionTitle                                         

C:\Users\Shabeeb\AppData\Local\Temp\ipykernel_15380\2492965747.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  document_section = df.loc[section_index]


In [13]:
COMPLETIONS_API_PARAMS = {
    # We use temperature of 0.0 because it gives the most predictable, factual answer.
    "temperature": 0.0,
    "max_tokens": 300,
    "model": COMPLETIONS_MODEL,
}

In [14]:
def answer_query_with_context(
    query: str,
    df: pd.DataFrame,
    document_embeddings: dict[(str, str), np.array],
    show_prompt: bool = False
) -> str:
    prompt = construct_prompt(
        query,
        document_embeddings,
        df
    )
    
    if show_prompt:
        print(prompt)

    response = openai.Completion.create(
                prompt=prompt,
                **COMPLETIONS_API_PARAMS
            )

    return response["choices"][0]["text"].strip(" \n")

In [15]:
answer_query_with_context('What is the difference between "fraudulently" and "defraud"?', df, document_embeddings)

C:\Users\Shabeeb\AppData\Local\Temp\ipykernel_15380\2492965747.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  document_section = df.loc[section_index]


41
196
155
177
Selected 3 document sections:
('SectionNumber: 25', '"Fraudulently".')
('SectionNumber: 207', 'Fraudulent claim to property to prevent its seizure as forfeited or in execution:')
('SectionNumber: 477', 'Fraudulent cancellation, destruction, etc., of will, authority to adopt, or valuable security:')


'"Fraudulently" refers to an act that is done with the intent to deceive or cheat, while "defraud" means to deprive someone of something through deception or fraud.'

In [16]:
query = "Is the public servant also liable to pay a fine?"
answer = answer_query_with_context(query, df, document_embeddings)

print(f"\nQ: {query}\nA: {answer}")

C:\Users\Shabeeb\AppData\Local\Temp\ipykernel_15380\2492965747.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  document_section = df.loc[section_index]


106
65
109
84
93
116
Selected 5 document sections:
('SectionNumber: 189', 'Threat of injury to public servant:')
('SectionNumber: 168', 'Public servant unlawfully engaging in trade:')
('SectionNumber: 179', 'Refusing to answer public servant authorised to question:')
('SectionNumber: 186', 'Obstructing public servant in discharge of public functions:')
('SectionNumber: 170', 'Personating a public servant:')

Q: Is the public servant also liable to pay a fine?
A: I don't know.


In [17]:
query = "129 Public servant "
answer = answer_query_with_context(query, df, document_embeddings)

print(f"\nQ: {query}\nA: {answer}")

C:\Users\Shabeeb\AppData\Local\Temp\ipykernel_15380\2492965747.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  document_section = df.loc[section_index]


1675
Selected 0 document sections:


Q: 129 Public servant 
A: I don't know.


In [18]:
query = "What is the right of private defence?"
answer = answer_query_with_context(query, df, document_embeddings)

print(f"\nQ: {query}\nA: {answer}")

C:\Users\Shabeeb\AppData\Local\Temp\ipykernel_15380\2492965747.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  document_section = df.loc[section_index]


158
272
32
90
Selected 3 document sections:
('SectionNumber: 106', 'Right of private defence against deadly assault when there is risk of harm to innocent person:')
('SectionNumber: 97', 'Right of private defence of the body and of property:')
('SectionNumber: 96', 'Things done in private defence:')

Q: What is the right of private defence?
A: The right of private defence is the right to protect oneself or one's property from harm or damage.
